In [20]:
import glob
import numpy as np
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import BootstrapFunctions as bsf

In [21]:
#List out the perturbation types: u is up, d is down, n is none. l for lambda, g for gamma.  Add 1,2 to balance the neutral points
pertcases = {"lngn":[0,1,2],"lngu":[3,4,5],"lngd":[6,7,8],
             "lugn":[9,10,11],"lugu":[12,13,14],"lugd":[15,16,17],
             "ldgn":[18,19,20],"ldgu":[21,22,23],"ldgd":[24,25,26]}
oneparamperts = pertcases["lngn"]+pertcases["lugn"]+pertcases["ldgn"]+pertcases["lngu"]+pertcases["lngd"]
no_nonperts = pertcases["lugn"]+pertcases["ldgn"]+pertcases["lngu"]+pertcases["lngd"]
inv_pertcases = {v_int:k for k, v in pertcases.items() for v_int in v}

In [22]:
paramlist = ['lambda','beta_x','gamma','beta_y','alpha','beta_z','nx','Kx','ny','Ky','offset']
byfilelist = ['fileN','fileda','filedb','sysname']

In [23]:
standard_columns = ['E[rhoxy]','E[CVx/CVy]','E[<x>]','E[<y>]','E[<F_x>]','E[<F_y>]',
                    '97.5% (rhoxy)','2.5% (rhoxy)','97.5% (CVx/CVy)','2.5% (CVx/CVy)',
                    '97.5% (<x>)','2.5% (<x>)','97.5% (<y>)','2.5% (<y>)',
                    '97.5% (<F_x>)','2.5% (<F_x>)','97.5% (<F_y>)','2.5% (<F_y>)',
                    'F_y at avg','F_x at avg','Var[lambda]','Var[gamma]','E[lambda]','E[gamma]',
                    'consistent fy signs (avg)','consistent fy signs (at avg)',
                    'consistent fx signs (avg)','consistent fx signs (at avg)']
parentfolders = ['IntHill','IntHill_consistent_violations_test*','IntHill_consistent_violator_1e8','IntHill_consistent_violator_largeN',
                 'IntHill_neg1fxoverlineattempts','IntHill_randomstart','IntHill_randomstart_highfx','IntHill_randomstart_nofb','IntHill_Violator_largeN_plusbonus']

In [24]:
def rate(x,y,alpha,nx,Kx,ny,Ky,o):
    return alpha*(y**ny)/(Ky**ny+y**ny) + ((x**nx)/(Kx**nx+x**nx)) + o
def rate_deriv_x(x,nx,Kx):
    return nx*(x**(nx-1))*(Kx**nx)/(Kx**nx+x**nx)**2
def rate_deriv_y(y,alpha,ny,Ky):
    return alpha*ny*(y**(ny-1))*(Ky**ny)/(Ky**ny+y**ny)**2
def sensitivty_x(x,y,alpha,nx,Kx,ny,Ky,o):
    return x*rate_deriv_x(x,nx,Kx)/rate(x,y,alpha,nx,Kx,ny,Ky,o)
def sensitivty_y(x,y,alpha,nx,Kx,ny,Ky,o):
    return y*rate_deriv_y(y,alpha,ny,Ky)/rate(x,y,alpha,nx,Kx,ny,Ky,o)

Notebook used to generate data for alldots.ipynb

In [25]:
no_feedback = []
just_mrna_feedback = []
just_protein_feedback = []
both_feedback = []
names = ['3Dno_feedback','3Djust_mrna_feedback','3Djust_protein_feedback','3Dboth_feedback']

In [26]:
#For outputting the standard form of the data
for parent in parentfolders:
    folders = glob.glob(f'Data/{parent}/*/*')
    dat = []
    failedsims = []
    bootstrapdicts = []
    for i,folder in enumerate(folders):
        files = glob.glob(folder+'/*data.csv',recursive=True)
        for j,f in enumerate(files):
            data = pd.read_csv(f)
            data = data.reset_index().rename(columns={'index':'perturbation'})
            numstrings = f.split("_")[-2].split('n')
            if (data[['Cov Bal xx','Cov Bal yy','Cov Bal xy','Cov Bal xz','Cov Bal yz','Cov Bal zz']] > 0.05).any(axis=1).any():
                failedsims.append(f)
                continue
            if 'offset' not in data:
                data['offset'] = 0
            dat.append(data)
            bsdict = bsf.pertexp_interpret(data,'all',analysed_columns=['<x>','<y>','<z>',"<F_x>","<F_y>"])
            fys = sensitivty_y(data['<x>'].astype(float),data['<y>'].astype(float),
                                            data['alpha'],data['nx'].astype(float),data['Kx'],
                                            data['ny'].astype(float),data['Ky'],data['offset'])
            fxs = sensitivty_x(data['<x>'].astype(float),data['<y>'].astype(float),
                                            data['alpha'],data['nx'].astype(float),data['Kx'],
                                            data['ny'].astype(float),data['Ky'],data['offset'])
            bsdict['F_y at avg'] = fys.mean()
            bsdict['F_x at avg'] = fxs.mean()
            bsdict['consistent fy signs (avg)'] = np.sum(np.sign(data['<F_y>']))
            bsdict['consistent fx signs (avg)'] = np.sum(np.sign(1-data['<F_x>']))
            bsdict['consistent fy signs (at avg)'] = np.sum(np.sign(fys))
            bsdict['consistent fx signs (at avg)'] = np.sum(np.sign(1-fxs))
            bsdict['fileN'] = float(numstrings[0])
            bsdict['fileda'] = float(numstrings[1])
            bsdict['filedb'] = float(numstrings[2])
            bsdict['sysname'] = "/".join(f.split("/")[:-1])
            for p in paramlist:
                try:
                    bsdict[p] = data[p][0]
                except KeyError:
                    bsdict[p] = 0
            if data['nx'][0] == 0 and data['ny'][0] == 0:
                no_feedback.append(bsdict)
            elif data['nx'][0] != 0 and data['ny'][0] == 0:
                just_mrna_feedback.append(bsdict)
            elif data['nx'][0] == 0 and data['ny'][0] != 0:
                just_protein_feedback.append(bsdict)
            else:
                both_feedback.append(bsdict)
    print(f"Finished {parent}")
for i,dataset in enumerate([no_feedback,just_mrna_feedback,just_protein_feedback,both_feedback]):
    df = pd.DataFrame(dataset)
    df.set_index(['fileN','fileda','filedb','sysname'],inplace=True)
    df[paramlist+['E[<F_x>]','E[<F_y>]','F_y at avg','F_x at avg','consistent fy signs (avg)','consistent fx signs (avg)',
                  'consistent fy signs (at avg)','consistent fx signs (at avg)']].to_csv(f'Data/parameters/{names[i]}.csv')

Finished IntHill
Finished IntHill_consistent_violations_test*
Finished IntHill_consistent_violator_1e8
Finished IntHill_consistent_violator_largeN
Finished IntHill_neg1fxoverlineattempts
Finished IntHill_randomstart
Finished IntHill_randomstart_highfx
Finished IntHill_randomstart_nofb
Finished IntHill_Violator_largeN_plusbonus
